In [ ]:
file_path = '/Users/angus/work/sukinda/lab_work/cations/raw_data/2025_01_09_Angus_River_waters.csv'
file_path = '/Users/angus/work/sukinda/lab_work/cations/raw_data/2025_01_13_ANGUS_RIVER_WATERS.csv'
%run main.py $file_path
%run unit_conv.py 'output.nc'

from utils import select_vars_by_element as sel_element
from utils import select_by_fuzzy_sample as search_samples
from utils import find_common_elements as common_elements
from utils import find_missing_elements as missing_elements
from utils import average_wavelengths

<xarray.Dataset>
Dimensions:                      (sample_name: 61, species: 39)
Coordinates:
  * sample_name                  (sample_name) object 'A' 'BW' 'B' ... 'T8' 'T9'
  * species                      (species) object 'Al 396.152 Ax' ... 'Sr 421...
Data variables: (12/29)
    Flags                        (sample_name, species) object '' '' ... '' ''
    Unadjusted_Data              (sample_name, species) float64 0.0027 ... 0....
    Concentration                (sample_name, species) float64 0.0027 ... 0....
    Intensity                    (sample_name, species) float64 108.0 ... 5.0...
    Intensity_SD                 (sample_name, species) float64 6.237 ... 105.1
    Intensity___RSD              (sample_name, species) float64 5.77 ... 0.21
    ...                           ...
    Flags_Replicate_1            (sample_name, species) object '' '' ... '' ''
    Flags_Replicate_2            (sample_name, species) object '' '' ... '' ''
    Flags_Replicate_3            (sample_nam

/Users/angus/work/icp_oes/main.py:188: UserWarning: rename 'sample_name' to 'sample_name' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds = ds.rename(coord_mapping)
/Users/angus/work/icp_oes/main.py:188: UserWarning: rename 'species' to 'species' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds = ds.rename(coord_mapping)


In [3]:
good_list = common_elements(good_wavelengths)
good_list = {'Al 396.152 R'}

In [4]:
avg_da = average_wavelengths(ds, good_list)

In [5]:
avg_da.sel(sample_name='SLRS-6_2')

<xarray.DataArray 'Concentration_mM' (species: 1)>
array([0.00150103])
Coordinates:
    sample_name  <U8 'SLRS-6_2'
  * species      (species) <U10 'Al_average'

In [6]:
import hvplot.xarray  # noqa
import xarray as xr

violin_plot = avg_da.hvplot.violin(y='Concentration_mM', by='species')
special_point = avg_da.sel(sample_name='SLRS-6_2')  
point_plot = special_point.hvplot.scatter(x='species', y='Concentration_mM', color='red',
                                           marker='*', size=15)
combined_plot = violin_plot * point_plot
combined_plot


:Overlay
   .Violin.I  :Violin   [species]   (Concentration_mM)
   .Scatter.I :Scatter   [species]   (Concentration_mM)

In [7]:
import hvplot.xarray
import holoviews as hv
hv.extension('bokeh')  # or your preferred backend

# Assuming your xarray DataArray is named avg_da and has coordinates 'species' and 'sample_name'
species_list = avg_da.coords['species'].values

plots = []

for sp in species_list:
    # Filter the data for the current species
    species_data = avg_da.sel(species=sp)
    
    # Create a violin plot for this species.
    # (Since no x is specified, hvPlot will plot a single violin centered at a default x value, e.g. 0.)
    violin = species_data.hvplot.hist(
        y='Concentration_mM',
        title=f"Species: {sp}",
        width=300,
        height=300,
        bins=30,
        xlim = (0, 0.001)
    ).opts(framewise=True)

    # Filter for the special sample points within this species.
    # (If the special samples don't exist for a given species, this selection may return an empty object.)
    special_point = species_data.sel(sample_name=['SLRS-6_1', 'SPS-SW2 10%_1'])
    
    # Create a scatter plot for the special points.
    # Use a constant x coordinate (here, 0) so that the points overlay the violin.
    special_scatter = special_point.hvplot.scatter(
        x='sample_name',   # force a constant x value
        y='Concentration_mM',
        color='red',
        marker='*',
        size=15
    ).opts(framewise=True)
    
    # Overlay the special points on the violin plot.
    combined = violin * special_scatter
    combined = violin 
    combined = combined.opts(framewise=True)
    plots.append(combined)

# Arrange all species plots into a grid with independent axes.
layout = hv.Layout(plots).cols(2).opts(shared_axes=False)
layout


:Layout
   .Histogram.I :Histogram   [Concentration_mM]   (Count)

In [12]:
from utils import subset_by_fuzzy
subset_by_fuzzy(ds, ['D', 'SLRS', 'SPS-SW2 10%'], threshold=0.4)

<xarray.Dataset>
Dimensions:                      (sample_name: 32, species: 39)
Coordinates:
  * sample_name                  (sample_name) object 'D10' ... 'SPS-SW2 10%_5'
  * species                      (species) object 'Al 396.152 Ax' ... 'Sr 421...
Data variables: (12/29)
    Flags                        (sample_name, species) object '' '' ... '' ''
    Unadjusted_Data              (sample_name, species) float64 0.0313 ... 0....
    Concentration                (sample_name, species) float64 0.0313 ... 0....
    Intensity                    (sample_name, species) float64 649.2 ... 6.5...
    Intensity_SD                 (sample_name, species) float64 24.88 ... 33.65
    Intensity___RSD              (sample_name, species) float64 3.83 ... 0.51
    ...                           ...
    Flags_Replicate_1            (sample_name, species) object '' '' ... '' ''
    Flags_Replicate_2            (sample_name, species) object '' '' ... '' ''
    Flags_Replicate_3            (sample_name, species) object '' '' ... '' ''
    Flags_Replicate_4            (sample_name, species) object '' '' ... '' ''
    Flags_Replicate_5            (sample_name, species) object '' '' ... '' ''
    Concentration_mM             (sample_name, species) float64 0.00116 ... 0...